# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 27

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)#, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [4]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [5]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [6]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [7]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [11]:
#metric = 'valuePoints metric'
metric = 'adjusted points per game'
#metric = 'form 5'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
411,Patrício,Wolves,51,27.000000,108,121.077666,4.0,4.484358,5.722111,5.074363,1.985707
340,Lloris,Tottenham,53,10.975610,45,47.826208,4.1,4.357499,5.038729,5.167056,1.892777
471,Henderson,Sheffield Utd,52,26.136364,115,108.307335,4.4,4.143933,5.138143,4.989944,1.817235
603,Reina,Aston Villa,45,5.000000,18,20.637434,3.6,4.127487,4.784726,4.627834,1.945716
93,Pope,Burnley,48,26.976744,116,108.773171,4.3,4.032109,5.209188,5.057648,1.840398
189,Alisson,Liverpool,62,19.130435,88,76.441352,4.6,3.995798,4.464270,4.800379,1.604751
341,Gazzaniga,Tottenham,43,16.764706,57,66.736502,3.4,3.980774,10.507274,5.355977,1.919700
168,Schmeichel,Leicester City,53,27.073171,111,106.340418,4.1,3.927889,6.225701,4.841448,1.706166
235,de Gea,Manchester Utd,54,26.764706,91,103.269754,3.4,3.858430,3.576968,3.359687,1.660404


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
182,Alexander-Arnold,Liverpool,77,27.166667,163,154.576234,6.0,5.689923,7.286739,6.394058,2.050506
181,Robertson,Liverpool,70,27.200000,136,135.913639,5.0,4.996825,5.806186,5.321339,1.888622
401,Doherty,Wolves,61,24.750000,99,111.862564,4.0,4.519700,4.998045,4.720334,1.829973
103,Alonso,Chelsea,60,9.016393,55,40.379918,6.1,4.478500,5.313052,5.569452,1.828340
297,Lundstram,Sheffield Utd,49,25.777778,116,114.877735,4.5,4.456464,2.563970,2.756615,2.013225
183,van Dijk,Liverpool,66,26.800000,134,116.881934,5.0,4.361266,4.756116,4.724007,1.697621
105,Azpilicueta,Chelsea,58,25.312500,81,110.236138,3.2,4.355008,4.343715,5.466811,1.808319
208,Otamendi,Manchester City,50,18.055556,65,73.926612,3.6,4.094397,4.352087,4.051855,1.831070
185,Matip,Liverpool,52,8.048780,33,32.628138,4.1,4.053799,2.830943,2.830943,1.777708


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,128,23.972603,175,167.026092,7.3,6.967374,6.802155,7.116795,1.947440
192,Mané,Liverpool,123,23.880597,160,146.634594,6.7,6.140324,4.365854,4.838869,1.750809
215,De Bruyne,Manchester City,108,26.176471,178,159.783455,6.8,6.104087,6.020280,5.632569,1.857415
618,Fernandes,Manchester Utd,82,3.013699,22,18.357609,7.3,6.091389,8.473466,8.220793,2.127205
214,Sterling,Manchester City,117,23.200000,116,128.583368,5.0,5.542387,4.835166,4.266669,1.620332
342,Son,Tottenham,99,21.034483,122,107.725123,5.8,5.121358,7.375337,6.008171,1.627675
217,Mahrez,Manchester City,85,21.818182,120,104.327945,5.5,4.781697,5.815161,5.362504,1.640109
431,Pulisic,Chelsea,69,15.909091,70,73.070130,4.4,4.592980,NaN,2.427305,1.748517
239,Martial,Manchester Utd,80,21.132075,112,96.974775,5.3,4.588985,3.952366,4.055970,1.622451


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
233,Rashford,Manchester Utd,89,21.967213,134,125.650046,6.1,5.719890,NaN,5.555840,1.917312
166,Vardy,Leicester City,97,25.161290,156,136.092056,6.2,5.408787,3.029474,3.239616,1.736657
210,Agüero,Manchester City,120,20.847458,123,109.068037,5.9,5.231719,4.630316,5.094550,1.510267
187,Firmino,Liverpool,97,26.808511,126,137.916074,4.7,5.144488,6.193767,6.180029,1.651796
409,Jiménez,Wolves,79,27.058824,138,137.772056,5.1,5.091576,4.865474,5.071484,1.811501
11,Aubameyang,Arsenal,109,25.000000,150,124.826028,6.0,4.993041,5.629913,4.668168,1.512349
460,Abraham,Chelsea,76,25.192308,131,123.210046,5.2,4.890780,3.421445,3.813247,1.774073
278,Pukki,Norwich City,65,26.170213,123,126.346028,4.7,4.827856,4.394737,4.676986,1.893641
313,Ings,Southampton,71,26.862745,137,125.102009,5.1,4.657082,3.328912,4.151129,1.747772


Below we compile a list sorting players based on a given metric (irrespective of position).

In [9]:
metric = 'adjusted points per game'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,128,23.972603,175,167.026092,7.3,6.967374,6.802155,7.116795,1.947440
192,Mané,Liverpool,123,23.880597,160,146.634594,6.7,6.140324,4.365854,4.838869,1.750809
215,De Bruyne,Manchester City,108,26.176471,178,159.783455,6.8,6.104087,6.020280,5.632569,1.857415
618,Fernandes,Manchester Utd,82,3.013699,22,18.357609,7.3,6.091389,8.473466,8.220793,2.127205
233,Rashford,Manchester Utd,89,21.967213,134,125.650046,6.1,5.719890,NaN,5.555840,1.917312
182,Alexander-Arnold,Liverpool,77,27.166667,163,154.576234,6.0,5.689923,7.286739,6.394058,2.050506
214,Sterling,Manchester City,117,23.200000,116,128.583368,5.0,5.542387,4.835166,4.266669,1.620332
166,Vardy,Leicester City,97,25.161290,156,136.092056,6.2,5.408787,3.029474,3.239616,1.736657
210,Agüero,Manchester City,120,20.847458,123,109.068037,5.9,5.231719,4.630316,5.094550,1.510267


In [10]:
metric = 'form 10'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
618,Fernandes,Manchester Utd,82,3.013699,22,18.357609,7.3,6.091389,8.473466,8.220793,2.127205
191,Salah,Liverpool,128,23.972603,175,167.026092,7.3,6.967374,6.802155,7.116795,1.947440
182,Alexander-Arnold,Liverpool,77,27.166667,163,154.576234,6.0,5.689923,7.286739,6.394058,2.050506
150,Richarlison,Everton,83,24.807692,129,113.704419,5.2,4.583434,6.449434,6.255538,1.590933
187,Firmino,Liverpool,97,26.808511,126,137.916074,4.7,5.144488,6.193767,6.180029,1.651796
342,Son,Tottenham,99,21.034483,122,107.725123,5.8,5.121358,7.375337,6.008171,1.627675
92,Vydra,Burnley,53,8.000000,24,21.202009,3.0,2.650251,6.921350,5.659858,1.151196
215,De Bruyne,Manchester City,108,26.176471,178,159.783455,6.8,6.104087,6.020280,5.632569,1.857415
103,Alonso,Chelsea,60,9.016393,55,40.379918,6.1,4.478500,5.313052,5.569452,1.828340
